# 직접(*)

In [20]:
import numpy as np

def hungarian_algorithm(cost_matrix):
    num_rows, num_cols = cost_matrix.shape
    num_workers = max(num_rows, num_cols)
    
    # Step 1: 입력 데이터 준비
    # 비용 행렬의 크기를 작업자 수와 작업 수 중 더 큰 값으로 설정
    cost_matrix = pad_matrix(cost_matrix, num_workers)
    
    # 초기화 단계
    row_reduction = np.zeros(num_workers)
    col_reduction = np.zeros(num_workers)
    assignment = np.zeros(num_workers, dtype=int)  # 할당 결과를 저장할 배열
    
    # Step 2: 초기화
    row_reduction, col_reduction, cost_matrix = initialize(cost_matrix, row_reduction, col_reduction)
    
    # Step 3: 대안 경로 찾기
    while True:
        marked_zeros = find_marked_zeros(cost_matrix)
        while marked_zeros:
            row, col = marked_zeros.pop()
            star_zeros, prime_zeros = find_star_prime_zeros(row, col, assignment)
            if len(star_zeros) == 0:
                step = 4
                break
            else:
                marked_zeros.extend(star_zeros)
                marked_zeros.extend(prime_zeros)
                assignment[row] = col
        if step == 4:
            break
        
        # Step 4: 경로 개수 확인
        marked_rows, marked_cols = mark_rows_cols(assignment)
        if len(marked_rows) + len(marked_cols) == num_workers:
            step = 6
            break
        
        # Step 5: 최소 비용 증가
        min_value = find_min_unmarked(cost_matrix, marked_rows, marked_cols)
        row_reduction, col_reduction = update_reductions(row_reduction, col_reduction, marked_rows, marked_cols, min_value)
        cost_matrix = update_cost_matrix(cost_matrix, marked_rows, marked_cols, min_value)
    
    # Step 6: 최적 결과 반환
    optimal_assignment = assignment[:num_rows]
    return assignment
    # return optimal_assignment


# 도움 함수들...

# 비용 행렬 크기를 맞추기 위해 작업자 수와 작업 수 중 더 큰 크기로 패딩(0)합니다.
def pad_matrix(cost_matrix, num_workers):
    num_rows, num_cols = cost_matrix.shape
    max_dim = max(num_rows, num_cols)
    padded_matrix = np.zeros((max_dim, max_dim))
    padded_matrix[:num_rows, :num_cols] = cost_matrix
    return padded_matrix

# 초기화 단계에서 행과 열의 최소값을 찾아서 각 행과 열의 원소에서 최소값을 뺍니다.
def initialize(cost_matrix, row_reduction, col_reduction):
    row_min = np.min(cost_matrix, axis=1)
    col_min = np.min(cost_matrix, axis=0)
    row_reduction += row_min
    col_reduction += col_min
    cost_matrix -= row_min.reshape((-1, 1))
    cost_matrix -= col_min
    return row_reduction, col_reduction,cost_matrix

def find_marked_zeros(matrix):
    marked_zeros = []
    num_rows, num_cols = matrix.shape
    for row in range(num_rows):
        for col in range(num_cols):
            if matrix[row, col] == 0:
                marked_zeros.append((row, col))
    return marked_zeros

def find_star_prime_zeros(row, col, assignment):
    num_workers = assignment.shape[0]
    star_zeros = []
    prime_zeros = []
    for i in range(num_workers):
        if assignment[i] == col:
            star_zeros.append((i, col))
        elif assignment[row] == i:
            prime_zeros.append((row, i))
    return star_zeros, prime_zeros

In [21]:
cost_matrix = np.array([
    [4, 2, 8, 5],
    [3, 9, 1, 6],
    [7, 5, 3, 4],
    [2, 6, 5, 3],
    [8, 1, 6, 4]
])

# 라이브러리

In [28]:
import numpy as np
from scipy.optimize import linear_sum_assignment

def hungarian_algorithm(cost_matrix):
    # 행렬의 크기
    rows, cols = cost_matrix.shape

    # 선형 합 최소화 문제로 변환하기 위해 가중치를 음수로 변경
    cost_matrix = -cost_matrix

    # linear_sum_assignment 함수를 사용하여 최적의 매칭을 찾음
    row_ind, col_ind = linear_sum_assignment(cost_matrix)

    # 매칭 결과 반환
    matches = []
    for row, col in zip(row_ind, col_ind):
        matches.append((row, col))

    return matches

# 예시로 사용할 가중치 행렬
cost_matrix = np.array([
    [1, 2, 10],
    [4, -5, 6],
    [7, 8, 9]
])

# 헝가리안 알고리즘 적용
matches = hungarian_algorithm(cost_matrix)

# 결과 출력
for row, col in matches:
    print(f"Row {row} - Column {col}")


Row 0 - Column 2
Row 1 - Column 0
Row 2 - Column 1
